In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import scipy.special
import numpy as np
import time
import math
import os
# from tqdm import tqdm
from tqdm.notebook import tqdm
import importlib

In [3]:
os.chdir("/Users/edgan/Documents/projects/fastlr")

In [4]:
df = pd.read_csv(
    "~/Documents/datasets/avazu/train.csv",
    nrows=1_000_000
)

In [5]:
target = "click"
CAT_COLS = [
    "C1", "banner_pos", 
    "site_category", "app_category", 
    "device_type", "device_conn_type",
#     "C14"
]
df_enc = pd.get_dummies(df[CAT_COLS], columns=CAT_COLS)
df_enc["c"] = 1
# df_final = pd.concat([
#     df[target], df_enc
# ], axis=1)

In [6]:
Xt, Xv, yt, yv = sklearn.model_selection.train_test_split(
    df_enc.values, df[target].values,
    random_state=0
)

In [7]:
n = len(Xt)
sizes = np.geomspace(100, n, 10).astype(int)
results = []
r = np.random.RandomState(0)
for size in tqdm(sizes):
    n_trials = 2
    for trial_idx in range(n_trials):
        s_idxs = r.choice(n, size=size, replace=False)
        Xs = Xt[s_idxs]
        ys = yt[s_idxs]
        sklm = sklearn.linear_model.LogisticRegression(
            max_iter=100,
            C=1,
            penalty="l2",
            solver="liblinear"
        )
        sklm.fit(Xs, ys)
        yh = sklm.predict_proba(Xt)
        score = sklearn.metrics.log_loss(yt, yh)
        results.append({
            "size": size,
            "trial_idx": trial_idx,
            "score": score,
        })
res_df = pd.DataFrame(results)

In [8]:
res_df.groupby(["size"]).aggregate({"score": ["mean"]})

,score
,mean
size,
100,0.495177
269,0.442556
726,0.432516
1957,0.428537
5275,0.427431
14217,0.426363
38315,0.425688
103261,0.425426


In [31]:
sample_size = 100
true_mask = yt==1
n_true = np.sum(true_mask)
r = np.random.RandomState(0)
s1_idxs = r.choice(np.argwhere(true_mask).ravel(), size=sample_size//2, replace=False)
s2_idxs = r.choice(np.argwhere(~true_mask).ravel(), size=sample_size//2, replace=False)
s_idxs = np.concatenate([s1_idxs, s2_idxs])

In [33]:
Xs = Xt[s_idxs]
ys = yt[s_idxs]
sklm = sklearn.linear_model.LogisticRegression(
    max_iter=100,
    C=1,
    penalty="l2",
    solver="liblinear",
    class_weight={0:(n-n_true)/len(s2_idxs), 1:(n_true)/len(s1_idxs)}
)
sklm.fit(Xs, ys)
yh = sklm.predict_proba(Xt)
score = sklearn.metrics.log_loss(yt, yh)
print("Score: {}".format(score))

Score: 0.6873454103226276
